In [9]:
import pandas as pd
import numpy as np
from sklearn import  preprocessing
import xgboost as xgb
import lightgbm as lgb    

df=pd.read_csv('train_ccf_first_round_user_shop_behavior.csv')
shop=pd.read_csv('train_ccf_first_round_shop_info.csv')
test=pd.read_csv('AB_test_evaluation_public.csv')
df=pd.merge(df,shop[['shop_id','mall_id']],how='left',on='shop_id')
train=pd.concat([df,test])
mall_list=list(set(list(shop.mall_id)))
result=pd.DataFrame()

In [10]:
train

,latitude,longitude,mall_id,row_id,shop_id,time_stamp,user_id,wifi_infos
0,32.088040,122.308291,m_1409,NaN,s_2871718,2017-08-06 21:20,u_376,b_6396480|-67|false;b_41124514|-86|false;b_287...
1,32.087970,122.308162,m_1409,NaN,s_2871718,2017-08-06 21:20,u_376,b_6396480|-67|false;b_56328155|-73|false;b_411...
2,40.638214,117.365255,m_4079,NaN,s_181637,2017-08-02 13:10,u_1041,b_8006367|-78|false;b_2485110|-52|false;b_3005...
3,31.197416,121.134451,m_6587,NaN,s_609470,2017-08-13 12:30,u_1158,b_26250579|-73|false;b_26250580|-64|false;b_26...
4,31.351320,122.255867,m_3005,NaN,s_3816766,2017-08-25 19:50,u_1654,b_39004150|-66|false;b_39004148|-58|false;b_21...
5,31.581963,122.595277,m_3839,NaN,s_3686420,2017-08-28 13:10,u_2733,b_13784698|-75|false;b_7390858|-86|false;b_358...
6,30.962847,104.815313,m_1920,NaN,s_89874,2017-08-11 14:20,u_2848,b_975810|-33|true;b_975807|-41|false;b_3506304...
7,39.589681,117.913713,m_2878,NaN,s_939447,2017-08-06 14:50,u_3063,b_15263246|-67|false;b_10120963|-76|false;b_41...
8,39.589699,117.913765,m_2878,NaN,s_939447,2017-08-02 17:30,u_3063,b_37749132|-77|false;b_25781349|-63|false;b_25...
9,39.589539,117.914182,m_2878,NaN,s_3698335,2017-08-02 18:30,u_3063,b_39846139|-73|false;b_25673942|-61|false;b_25...


In [11]:
def add_date_features(df):
    df["date"] = pd.to_datetime(df["time_stamp"])
    df["date_year"] = df["date"].dt.year
    df["date_month"] = df["date"].dt.month
    df["date_day"] = df["date"].dt.day
    df["date_hour"] = df["date"].dt.hour
    df["date_min"] = df["date"].dt.minute
    return df

In [12]:
add_date_features(train)
lb = preprocessing.LabelBinarizer()
#user_shop_info = user_shop_info[user_shop_info.wifi_flag_2.isin(['false', 'true'])]
train['longitude'] = train['longitude'].astype(np.float32)
train['latitude'] = train['latitude'].astype(np.float32)
train = train.drop(['date', 'time_stamp', 'date_year', 'date_month'], axis=1)
train

,latitude,longitude,mall_id,row_id,shop_id,user_id,wifi_infos,date_day,date_hour,date_min
0,32.088039,122.308289,m_1409,NaN,s_2871718,u_376,b_6396480|-67|false;b_41124514|-86|false;b_287...,6,21,20
1,32.087971,122.308159,m_1409,NaN,s_2871718,u_376,b_6396480|-67|false;b_56328155|-73|false;b_411...,6,21,20
2,40.638214,117.365257,m_4079,NaN,s_181637,u_1041,b_8006367|-78|false;b_2485110|-52|false;b_3005...,2,13,10
3,31.197416,121.134453,m_6587,NaN,s_609470,u_1158,b_26250579|-73|false;b_26250580|-64|false;b_26...,13,12,30
4,31.351320,122.255867,m_3005,NaN,s_3816766,u_1654,b_39004150|-66|false;b_39004148|-58|false;b_21...,25,19,50
5,31.581963,122.595276,m_3839,NaN,s_3686420,u_2733,b_13784698|-75|false;b_7390858|-86|false;b_358...,28,13,10
6,30.962847,104.815315,m_1920,NaN,s_89874,u_2848,b_975810|-33|true;b_975807|-41|false;b_3506304...,11,14,20
7,39.589680,117.913712,m_2878,NaN,s_939447,u_3063,b_15263246|-67|false;b_10120963|-76|false;b_41...,6,14,50
8,39.589699,117.913765,m_2878,NaN,s_939447,u_3063,b_37749132|-77|false;b_25781349|-63|false;b_25...,2,17,30
9,39.589539,117.914185,m_2878,NaN,s_3698335,u_3063,b_39846139|-73|false;b_25673942|-61|false;b_25...,2,18,30


In [14]:
weekend = [4, 5, 6, 11, 12, 13, 18, 19, 20, 25, 26, 27]
is_weekend = []
is_morning = []
is_afternoon = []
is_evening = []
is_dinner = []
for line in train.values:
    if line[-3] in weekend:
        is_weekend.append(1)
    else:
        is_weekend.append(0)

for line in train.values:
    if (line[-2] >= 0) & (line[-2] <= 8):
        is_morning.append(1)
    else:
        is_morning.append(0)
        
for line in train.values:
    if (line[-2] >= 9) & (line[-2] <= 16):
        is_afternoon.append(1)
    else:
        is_afternoon.append(0)
        
for line in train.values:
    if (line[-2] >= 17) & (line[-2] <= 23):
        is_evening.append(1)
    else:
        is_evening.append(0)

for line in train.values:
    if ((line[-2] >= 11) & (line[-2] <= 13)) | ((line[-2] >= 17) & (line[-2] <= 19)):
        is_dinner.append(1)
    else:
        is_dinner.append(0)
        
train['is_weekend'] = is_weekend
train['is_morning'] = is_morning
train['is_afternoon'] = is_afternoon
train['is_evening'] = is_evening
train['is_dinner'] = is_dinner
train

,latitude,longitude,mall_id,row_id,shop_id,user_id,wifi_infos,date_day,date_hour,date_min,is_weekend,is_morning,is_afternoon,is_evening,is_dinner
0,32.088039,122.308289,m_1409,NaN,s_2871718,u_376,b_6396480|-67|false;b_41124514|-86|false;b_287...,6,21,20,1,0,0,1,0
1,32.087971,122.308159,m_1409,NaN,s_2871718,u_376,b_6396480|-67|false;b_56328155|-73|false;b_411...,6,21,20,1,0,0,1,0
2,40.638214,117.365257,m_4079,NaN,s_181637,u_1041,b_8006367|-78|false;b_2485110|-52|false;b_3005...,2,13,10,0,0,1,0,1
3,31.197416,121.134453,m_6587,NaN,s_609470,u_1158,b_26250579|-73|false;b_26250580|-64|false;b_26...,13,12,30,1,0,1,0,1
4,31.351320,122.255867,m_3005,NaN,s_3816766,u_1654,b_39004150|-66|false;b_39004148|-58|false;b_21...,25,19,50,1,0,0,1,1
5,31.581963,122.595276,m_3839,NaN,s_3686420,u_2733,b_13784698|-75|false;b_7390858|-86|false;b_358...,28,13,10,0,0,1,0,1
6,30.962847,104.815315,m_1920,NaN,s_89874,u_2848,b_975810|-33|true;b_975807|-41|false;b_3506304...,11,14,20,1,0,1,0,0
7,39.589680,117.913712,m_2878,NaN,s_939447,u_3063,b_15263246|-67|false;b_10120963|-76|false;b_41...,6,14,50,1,0,1,0,0
8,39.589699,117.913765,m_2878,NaN,s_939447,u_3063,b_37749132|-77|false;b_25781349|-63|false;b_25...,2,17,30,0,0,0,1,1
9,39.589539,117.914185,m_2878,NaN,s_3698335,u_3063,b_39846139|-73|false;b_25673942|-61|false;b_25...,2,18,30,0,0,0,1,1


In [15]:
train = train.drop(['date_day', 'date_hour', 'date_min'], axis=1)

In [18]:
count = 0
for mall in mall_list:
    count = count + 1
    print(count)
    train1=train[train.mall_id==mall].reset_index(drop=True)       
    l=[]
    wifi_dict = {}
    for index,row in train1.iterrows():
        wifi_list = [wifi.split('|') for wifi in row['wifi_infos'].split(';')]
        for i in wifi_list:
            row[i[0]]=int(i[1])
            if i[0] not in wifi_dict:
                wifi_dict[i[0]]=1
            else:
                wifi_dict[i[0]]+=1
        l.append(row)    
    delate_wifi=[]
    for i in wifi_dict:
        if wifi_dict[i]<20:
            delate_wifi.append(i)
    m=[]
    for row in l:
        new={}
        for n in row.keys():
            if n not in delate_wifi:
                new[n]=row[n]
        m.append(new)
    train1=pd.DataFrame(m)
    df_train=train1[train1.shop_id.notnull()]
    df_test=train1[train1.shop_id.isnull()]
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(df_train['shop_id'].values))
    df_train['label'] = lbl.transform(list(df_train['shop_id'].values))    
    num_class=df_train['label'].max()+1    
    params = {
            'objective': 'multi:softmax',
            'eta': 0.15,
            'max_depth': 10,
            'eval_metric': 'merror',
            'seed': 0,
            'missing': -999,
            'num_class':num_class,
            'silent' : 1
            }
    feature=[x for x in train1.columns if x not in ['user_id','label','shop_id','time_stamp','mall_id','wifi_infos']]    
    xgbtrain = xgb.DMatrix(df_train[feature], df_train['label'])
    xgbtest = xgb.DMatrix(df_test[feature])
    watchlist = [ (xgbtrain,'train'), (xgbtrain, 'test') ]
    num_rounds=60
    model = xgb.train(params, xgbtrain, num_rounds, watchlist, early_stopping_rounds=15)
    df_test['label']=model.predict(xgbtest)
    df_test['shop_id']=df_test['label'].apply(lambda x:lbl.inverse_transform(int(x)))
    r=df_test[['row_id','shop_id']]
    result=pd.concat([result,r])
    result['row_id']=result['row_id'].astype('int')
    result.to_csv('sub_week_feature1.csv',index=False)

1


c:\users\scarlet\appdata\local\programs\python\python35\lib\site-packages\ipykernel\__main__.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


[0]	train-merror:0.069284	test-merror:0.069284
Multiple eval metrics have been passed: 'test-merror' will be used for early stopping.

Will train until test-merror hasn't improved in 15 rounds.
[1]	train-merror:0.060504	test-merror:0.060504
[2]	train-merror:0.052379	test-merror:0.052379
[3]	train-merror:0.047838	test-merror:0.047838
[4]	train-merror:0.044305	test-merror:0.044305
[5]	train-merror:0.041126	test-merror:0.041126
[6]	train-merror:0.0382	test-merror:0.0382
[7]	train-merror:0.035676	test-merror:0.035676
[8]	train-merror:0.033708	test-merror:0.033708
[9]	train-merror:0.031286	test-merror:0.031286
[10]	train-merror:0.029167	test-merror:0.029167
[11]	train-merror:0.026947	test-merror:0.026947
[12]	train-merror:0.02518	test-merror:0.02518
[13]	train-merror:0.023364	test-merror:0.023364
[14]	train-merror:0.022254	test-merror:0.022254
[15]	train-merror:0.02079	test-merror:0.02079
[16]	train-merror:0.019831	test-merror:0.019831
[17]	train-merror:0.019175	test-merror:0.019175
[18]	tr

c:\users\scarlet\appdata\local\programs\python\python35\lib\site-packages\ipykernel\__main__.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\users\scarlet\appdata\local\programs\python\python35\lib\site-packages\ipykernel\__main__.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


2
[0]	train-merror:0.308511	test-merror:0.308511
Multiple eval metrics have been passed: 'test-merror' will be used for early stopping.

Will train until test-merror hasn't improved in 15 rounds.
[1]	train-merror:0.234229	test-merror:0.234229
[2]	train-merror:0.191116	test-merror:0.191116
[3]	train-merror:0.156588	test-merror:0.156588
[4]	train-merror:0.129713	test-merror:0.129713
[5]	train-merror:0.103583	test-merror:0.103583
[6]	train-merror:0.088466	test-merror:0.088466
[7]	train-merror:0.076334	test-merror:0.076334
[8]	train-merror:0.066256	test-merror:0.066256
[9]	train-merror:0.058417	test-merror:0.058417
[10]	train-merror:0.049645	test-merror:0.049645
[11]	train-merror:0.044233	test-merror:0.044233
[12]	train-merror:0.038634	test-merror:0.038634
[13]	train-merror:0.035274	test-merror:0.035274
[14]	train-merror:0.033221	test-merror:0.033221
[15]	train-merror:0.030049	test-merror:0.030049
[16]	train-merror:0.026502	test-merror:0.026502
[17]	train-merror:0.024263	test-merror:0.0242

In [19]:
result

,row_id,shop_id
19817,118780,s_3360283
19818,118795,s_390525
19819,118801,s_466036
19820,118855,s_3507512
19821,118875,s_475860
19822,118996,s_565673
19823,119146,s_3029347
19824,119147,s_299684
19825,119194,s_209855
19826,119211,s_836849
